In [1]:
import sys
import geopandas
import pandas as pd
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

In [ ]:
grid_raw = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_raw.shp')

In [ ]:
grid_raw

In [4]:
# I.
grid_raw = grid_raw[~((grid_raw['left'] < 356500) & (grid_raw['top'] > 427300))]

In [5]:
# II.
grid_raw = grid_raw[~((grid_raw['left'] < 520000) & (grid_raw['top'] > 470000))]

In [6]:
# III.
grid_raw = grid_raw[grid_raw['top'] < 552500]

In [7]:
# IV.
grid_raw = grid_raw[~((grid_raw['right'] > 583000) & (grid_raw['top'] > 495000))]

In [8]:
# V.
grid_raw = grid_raw[~((grid_raw['right'] > 484500) & (grid_raw['bottom'] < 351000))]

In [9]:
# VI.
grid_raw = grid_raw[~((grid_raw['right'] > 196000) & (grid_raw['bottom'] < 350000) & (grid_raw['left'] < 372000))]

In [10]:
# VII.
grid_raw = grid_raw[~((grid_raw['left'] < 196000) & (grid_raw['bottom'] < 337000))]

In [11]:
# VIII.
grid_raw = grid_raw[~((grid_raw['right'] > 452500) & (grid_raw['bottom'] < 408000) & (grid_raw['left'] < 577500) & (grid_raw['top'] > 370000))]

In [12]:
grid_raw.shape

(5635990, 6)

In [31]:
grid_raw.shape

(5671876, 6)

In [13]:
# save to file
grid_raw.to_file("/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_raw_filtered.shp")

In [2]:
grid_raw = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_raw_man_filter.shp')

In [3]:
grid_raw.shape[0]

959189

In [4]:
path = "/home/philipp/Data/edin_diss/GIS_grid/parts_raw_4/"

n = 4
part = int(round(grid_raw.shape[0]/n,0))

for i in range(n):
    if i == n-1:
        grid_raw.iloc[i*part:].to_file(path + "GIS_grid_raw_" + str(i) + ".shp")
    else:
        grid_raw.iloc[i*part:(i+1)*part].to_file(path + "GIS_grid_raw_" + str(i) + ".shp")

In [11]:
grid_raw.iloc[:part].to_file("/home/philipp/Data/edin_diss/GIS_grid/parts/GIS_grid_raw_filtered_0.shp")

In [12]:
grid_raw.iloc[part:2*part].to_file("/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_raw_filtered_1.shp")
grid_raw.iloc[2*part:3*part].to_file("/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_raw_filtered_2.shp")
grid_raw.iloc[3*part:4*part].to_file("/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_raw_filtered_3.shp")
grid_raw.iloc[4*part:].to_file("/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_raw_filtered_4.shp")

In [5]:
grid_raw.iloc[:100].to_file("/home/philipp/Data/edin_diss/GIS_grid/parts_raw_4/GIS_grid_raw_100.shp")

In [6]:
959189/100/60/4

39.966208333333334

In [ ]:
# delete not needed attributes from cilper

In [2]:
grid_raw = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_grid/GIS_grid_dissolved_wo.shp')

In [3]:
grid_raw.columns

Index(['FB_KURZ', 'FB_LANG', 'FR_KURZ', 'FR_LANG', 'ABTEILUNG', 'UNTERABTEI',
       'FARBCODE', 'LINKID', 'SHAPE_Leng', 'SHAPE_Area', 'year_data', 'id',
       'FB', 'FR', 'Abt', 'UAbt', 'TFl', 'Esit', 'Bwirt', 'SWKat', 'Stoe',
       'Vtyp', 'Wgeb', 'year_start', 'year_end', 'geometry'],
      dtype='object')

In [4]:
grid_raw = grid_raw.drop(columns=['FB_KURZ', 'FB_LANG', 'FR_KURZ', 'FR_LANG', 'ABTEILUNG', 'UNTERABTEI', \
                                  'FARBCODE', 'LINKID', 'id', 'FB', 'FR', 'Abt', 'UAbt', 'TFl', 'Esit', \
                                  'Bwirt', 'SWKat', 'Stoe', 'Vtyp', 'Wgeb', 'year_start', 'year_end'])

In [5]:
grid_raw

,SHAPE_Leng,SHAPE_Area,year_data,geometry
0,1490.713604,31382.525999,2019,"MULTIPOLYGON (((426689.258 291445.923, 426693...."


In [6]:
grid_raw.to_file("/home/philipp/Data/edin_diss/GIS_grid/parts_raw_4/GIS_grid_dissolved_wo_clean.shp")